In [332]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# 기본 scheme 대신 seaborn scheme 세팅, 통일된 그림
# 일일이 그래프의 font size를 지정할 필요 없이 seaborn의 font_scale 사용하여 편함

import missingno as msno
# 데이터셋의 채워지지 않은 null data를 쉽게 보여줄 수 있는 라이브러리

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
# matplotlib으로 그린 다음 show를 하면 원래 새로운 창이 뜨면서 그림이 보이는데 
# inline으로 하면 이 노트북에서 바로 볼 수 있다.

#### Libraries
- numpy : numerical method에 대한 라이브러리  
- pandas : sql같은 데이터 프레임을 쉽게 다룰 수 있게 해준다.  
- matplotlib, seaborn : data visualization (ex. graph)  
 

#### 앞으로 우리가 해야할 프로세스는 대략 아래와 같습니다.
1. **데이터셋 확인** - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
2. **탐색적 데이터 분석(exploratory data analysis)** - 각각의 column, 즉 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.
3. **feature engineering** - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
4. **model 만들기** - sklearn이라는, 머신러닝에 필수적인 라이브러리를 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.
5. **모델 학습 및 예측** - trainset 을 가지고 모델을 학습시킨 후, testset 을 가지고 prediction 합니다.
6. **모델 평가** - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는 지 확인해봅니다.

# 1. Dataset 확인

#### pandas
- 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리
- 사용하여 데이터셋의 간단한 통계적 분석 부터, 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있다.
- 파이썬으로 데이터분석을 한다고 하면 반드시 능숙해져야 할 라이브러리 -> 반복해서 연습!!<br/><br/>

- 캐글에서 데이터셋은 보통 train set, test set 으로 나뉘어 있습니다.
- 비어져있는 null data가 있는지 확인해줘야 합니다.

In [333]:
#Loading csv files
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [334]:
df_train.head() # 데이터 셋의 처음 부분을 출력해줌, ()안의 개수만큼 (default = 5)

이렇게 엑셀처럼 테이블화되어있는 데이터를 쉽게 다루게 해주는 것이 pandas이다.  
- PassengerId : 1, 2, 3, ...
- Survived : 생존여부 / target label (0:dead, 1:alive) / int
- Pclass : 티켓의 클래스 / categorical feature (First class, second class, third class), 순서 유 / int
- Name : 이름
- Sex : 성별 / binary (male, female) -> 나중에 수치화하여 바꿀 것/ str
- Age : 나이 / continuous / int
- SibSp : 함께 탑승한 형제와 배우자의 수 / quantitative (몇 개, 몇 명 같은) / int
- Parch : 함께 탑승한 부모, 아이의 수 / quantitative / int
- Ticket : 티켓 번호 / 하나의 string 데이터 (alphabet + number) / str
- Fare : 탑승료 / continuous / float
- Cabin : 객실 번호 / 카테고리 분류 (alphabet + number) / str
- Embarked : 탑승 항구 / 카테고리(S,C,Q 3가지), 순서 무 / str

#### 데이터 타입마다 데이터를 처리하는 방식이 조금씩 달라진다.
##### 이 문제에서
- feature : **Pclass, Age, SibSp, Parch, Fare**
- 예측하려는 target label : **Survived**

In [335]:
df_train.describe() # 각 featurn/column의 간단한 통계적 수치를 알려준다. (ex. min, mean, etc.)

In [336]:
# 영상 추가 내용
df_train.shape

891명이 있음을 볼 수 있다.  
null값의 존재 여부는 표에서도 볼 수 있는데, Pclass, PassengerId 등의 경우 count로 보아 891명이나, Age의 경우 count가 714이다.  
따라서 Age에 null data가 있음을 확인할 수 있다.  
이는 1.1에서 문법으로도 확인할 수 있다.

In [337]:
df_test.describe()

테이블을 보면, count가 다 동일한 값이 아니다.  
따라서 숫자가 PassenserID 숫자와 다른 경우, null 값이 존재한다고 볼 수 있다. (null data가 존재하는 열(feature) 존재)  
이를 좀 더 보기 편하도록 그래프로 시각화해서 살펴보자. 

## 1.1 Null data check

### 방법 1 - 숫자 퍼센티지로 확인
각 데이터마다 null data가 얼마나 있는지 확인할 수 있다.

In [338]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100*(df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)
    
    # 각 컬럼마다 null 값의 퍼센트 알아보기 위해
    
    # {:>10} -> format문, 오른쪽으로 정렬
    '''df_train[col].isnull().sum()에서: 
            .isnull() -> null인가 (null이면 True, 아니면 False)
            .sum() -> 앞의 결과에서 (False:0, True:1)이므로 합은 null인 값의 개수를 알려줌] '''
    # df_train[col].shape[0] -> 전체로 나눠줘야하기 때문에 추가됨. 둘을 나누면 퍼센트로 구할 수 있다.

In [339]:
# 영상 추가 내용
df_train[col]

pandas의 기본 객체는 series이고, index와 value로 이루어져있다.  
이것이 여러 개 묶이면 하나의 dataframe을 만들어준다.

In [340]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100*(df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

Train, Test set 에서 Age(둘다 약 20%), Cabin(둘다 약 80%), Embarked(Train만 0.22%) null data 존재  
null data를 채울 수도, 안 채울 수도 있는데 성능에 따라 상황에 따라 다르게 한다. (이는 나중에)  
보통은 채우는 편이 성능이 더 잘 나온다.  

MSNO 라는 라이브러리를 사용하면 null data의 존재를 더 쉽게 볼 수 있다.

### 방법 2 - msno 사용

In [341]:
msno.matrix(df = df_train.iloc[:,:], figsize = (8,8), color=(0.8, 0.5, 0.2))

# msno는 아래와 같은 matrix 형태를 만들어준다.
# df_train.iloc[:,:] -> 모든 행, 열 선택 
# iloc = 'index location'으로, 원하는 위치의 데이터를 indexing하는 것 (아래 예시 참고)
# figsize = (8,8) -> 이건 그냥 matrix 이미지의 크기

빈 칸이 null이다.

In [342]:
# 영상 추가 내용
df_train.iloc[3:5, 2:]

In [343]:
# 바 그래프로도 표현할 수 있다.
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color = (0.8, 0.5, 0.2))

#### 위 두 가지 MSNO 방법 비교
- matrix - 어느 위치에 null 있는지, 분포 확인 가능  
- bar - 퍼센티지, 개수 등 숫자와 함께 확인 가능

In [344]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8), color=(0.8,0.5,0.2))

## 1.2 Target label 확인
Target label이 어떤 distribution을 가지고 있는지 확인해봐야 한다.   
Target label이 얼마나 밸런스 있게 data set이 있느냐가 몹시 중요하다.  
Binary classification 문제의 경우, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라질 수 있다.

In [345]:
f, ax = plt.subplots(1, 2, figsize = (18,8))
# 그래프를 그릴 때는 항상, 팔레트를 먼저 준비하고 그 위에 그린다. 이 코드는 도화지를 준비하는 과정이다.
# subplots -> 큰 틀이 있으면 그 부분부분을 의미, 여기서 1->row, 2->column이므로 1개의 행에 2개의 파트로 이루어진 그래프를 준비하겠다
# figsize = (18,8) -> 가로로 18, 세로로 8

df_train['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct = '%1.1f%%', ax = ax[0], shadow = True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data = df_train, ax = ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

'''df_train['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct = '%1.1f%%', ax = ax[0], shadow = True) 에서:
        df_train['Survived'].value_counts() -> Survived의 value를 count 하는 것 (아래 예시 참고)
        .pie -> pie plot으로 그림
        explode = [0, 0.1] -> 보기 힘드니까 두 색 사이를 째는 것
        autopct = '%1.1f%%' -> 퍼센트를 표시해준다.
        ax = ax[0] -> 도화지 어느 위치에 그릴 것이냐 (0 -> 첫 번째에 그리겠다.)
'''
# ax[0].set_title('Pie plot - Survived') -> ax[0]의 제목
# ax[0].set_ylabel('') -> ax[0]의 y label을 없애겠다.

안타깝게도 죽은 사람이 많습니다. 38.4 % 가 살아남았습니다.  

Target label 의 분포가 제법 균일(balanced)합니다. (값을 비교했을 때 극단적이지 않으므로 balance하다.)  
불균일한 경우, 예를 들어서 100중 1이 99, 0이 1개인 경우에는 만약 모델이 모든것을 1이라 해도 정확도가 99%가 나오게 된다.  
0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없게 된다. 지금 문제에서는 그렇지 않으니 계속 진행하였다.

In [346]:
# 영상 추가 내용
df_train['Survived'].value_counts()

# 'Survived'의 value를 각 레이블별로 count하는 것이다.
# type은 series이다.
# 그리고 series들은 plot을 가지고 있다.

In [347]:
df_train['Survived'].value_counts().plot()

In [348]:
# 이전과 같은 코드, 다른 형태
plt.plot(df_train['Survived'].value_counts())

# 2. Exploratory data analysis

이제 본격적인 데이터 분석  
어떤 feature가 중요한지, 상관관계 등을 확인해보는 작업  
-> 실제 모델에 어떤 값을 쓸지에 대한 인사이트를 얻을 수 있어 몹시 중요한 단계이다.  
특히 통계적인 인사이트가 중요하다.  

데이터가 매우 많아 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요하다.  
시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있고, 특정 목적에 맞는 소스코드를 정리해두어 필요할 때마다 참고하면 편하다.  

## 2.1 Pclass
- ordinal, 서수형 데이터
- 카테고리
- 순서가 있는 데이터 타입

먼저 Pclass 에 따른 생존률의 차이를 살펴보았다.  
엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe 에서는 groupby 를 사용하면 쉽게 할 수 있다. 또한 pivot 이라는 메소드도 있다.  
'Pclass', 'Survived' 를 가져온 후, pclass 로 묶는다. 그러고 나면 각 pclass 마다 0, 1 이 count가 되는데, 이를 평균내면 각 pclass 별 생존률이 나온다.  
아래와 같이 count() 를 하면, 각 class 에 몇명이 있는 지 확인할 수 있으며, sum() 을 하면, 216 명중 생존한(survived=1)사람의 총합을 주게 된다.  

In [349]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).count()

# ['Pclass', 'Survived'] -> 이렇게 리스트로 꼭 한 번 더 묶어야한다.

In [350]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).sum()

.count() -> sample이 총 몇 명인지  
.sum() -> 0과 1로 이루어져있으므로, 1이 몇 명인지

In [351]:
# 영상 추가 내용
df_train[['Pclass', 'Survived']]

pandas 의 crosstab 을 사용하면 좀 더 위 과정을 좀 더 수월하게 볼 수 있다.

In [352]:
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins = True).style.background_gradient(cmap='summer_r')

grouped 객체에 mean() 을 하게 되면, 각 클래스별 생존률을 얻을 수 있다. class 1 이면 아래와 같다.  

In [353]:
# 영상 추가 내용
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins = True)

# margins = True -> 아래 표에서 ALL이 있느냐 없느냐의 차이

In [354]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).mean().sort_values(by = 'Survived', ascending = False).plot.bar()

보다시피, Pclass 가 좋을 수록(1st) 생존률이 높은 것을 확인할 수 있다.  
좀 더 보기 쉽게 그래프를 그려보겠습니다. seaborn 의 countplot 을 이용하면, 특정 label 에 따른 개수를 확인해볼 수 있습니다.

In [355]:
# 영상 추가 내용
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).mean()

visualization을 하면 똑같은 그림이지만 다양하게 그릴 수 있다.  
결국 중요한 것은 다양하게 쓰는 게 좋다는 것이다. 더 나은 그림을 그리기 위해! 

아래 그래프는, 위와 같은 내용을 다르게 그린 것이다.

In [356]:
y_position = 1.02
f, ax = plt.subplots(1,2,figsize=(18,8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax = ax[0])
ax[0].set_title('Number of Passengers By Pclass', y = y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax = ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y = y_position)
plt.show()

이전에는 ax[1]의 0과 1을 ax[0]과 같이 묶어서 합쳐 표현했다면,  
0과 1의 색을 구분하여 각 클래스 별로 survived & dead를 비교할 수 있게 되었다.

클래스가 높을 수록, 생존 확률이 높은걸 확인할 수 있습니다. Pclass 1, 2, 3 순서대로 63%, 48%, 25% 입니다.  
우리는 생존에 Pclass 가 큰 영향을 미친다고 생각해볼 수 있으며, 나중에 모델을 세울 때 이 feature 를 사용하는 것이 좋을 것이라 판단할 수 있습니다.

## 2.2 Sex
이번에는 성별로 생존률이 어떻게 달라지는 지 확인해보겠습니다.  
마찬가지로 pandas groupby 와 seaborn countplot 을 사용해서 시각화해봅시다.

In [357]:
f, ax = plt.subplots(1,2,figsize=(18,8))
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index = True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue = 'Survived', data = df_train, ax = ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

'''df_train[['Sex', 'Survived']].groupby(['Sex'], as_index = True).mean().plot.bar(ax=ax[0])에서:
        df_train의 ['Sex', 'Survived'] column을 가져오고, 
        groupby를 해서 마치 피벗처럼
        as_index를 넣어 성별이 컬럼으로 가게
        이것의 평균을 bar graph로 plot한다.
        위치는 첫 번째!
'''

보시다시피, 여자가 생존할 확률이 높습니다.

In [358]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

이제 pandas의 crosstab을 이용해 더 보기 쉽게 만들어보겠습니다.

In [359]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins = True).style.background_gradient(cmap='summer_r')

Pclass 와 마찬가지로, Sex 도 예측 모델에 쓰일 중요한 feature 임을 알 수 있습니다

## 2.3 Both Sex and Pclass
이번에는 Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 봅시다.  
seaborn 의 factorplot 을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있습니다. 

In [360]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', data = df_train, size = 6, aspect = 1.5)

모든 클래스에서 female 이 살 확률이 male 보다 높은 걸 알 수 있습니다.  
또한 남자, 여자 상관없이 클래스가 높을 수록 살 확률 높습니다.  
위 그래프는 hue 대신 column 으로 하면 아래와 같아집니다.

In [361]:
sns.factorplot(x = 'Sex', y = 'Survived', col = 'Pclass',
              data = df_train, saturation = .5, size = 9, aspect = 1)

## 2.4 Age
이번에는 Age feature 를 살펴봅시다.

In [362]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

생존에 따른 Age의 histogram 을 그려보겠습니다.

In [363]:
fig, ax = plt.subplots(1,1,figsize = (9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax = ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax = ax)
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

# kdeplot() -> kernel density estimate plot, 아래 설명 참고

보시다시피, 생존자 중 나이가 어린 경우가 많음을 볼 수 있습니다.

(영상 추가 내용)
#### seaborn.kdeplot()
히스토그램  
현재 데이터들의 분포가 어떻게 되어있는지 estimation(추정)  
아래의 히스토그램 그래프와 비슷한데, 너무 딱딱하니까 하나의 스무스한 plot으로 만들어준다.  

In [364]:
df_train[df_train['Survived'] == 1]['Age'].hist()

이제 나이 말고 클래스별로 어떻게 나오는지 그려보도록 하겠습니다.

In [365]:
# Age distribution within classes
plt.figure(figsize = (8,6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind = 'kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

Class 가 높을 수록 나이 많은 사람의 비중이 커진다.  
나이대가 변하면서 생존률이 어떻게 되는 지 보려고 합니다.  
나이범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 봅시다.

In [366]:
cummulate_survival_ratio = []
for i in range(1,80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / len(df_train[df_train['Age'] < i]['Survived']))

plt.figure(figsize = (7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

보시다시피, 나이가 어릴 수록 생존률이 확실히 높은것을 확인할 수 있습니다.  
우리는 이 나이가 중요한 feature 로 쓰일 수 있음을 확인했습니다.

## 2.5 Pclass, Sex, Age
지금까지 본, Sex, Pclass, Age, Survived 모두에 대해서 보고싶습니다. 이를 쉽게 그려주는 것이 seaborn 의 violinplot 입니다.  
x 축은 우리가 나눠서 보고싶어하는 case(여기선 Pclass, Sex) 를 나타내고, y 축은 보고 싶어하는 distribution(Age) 입니다.  

In [367]:
f, ax = plt.subplots(1,2,figsize = (18,8))
sns.violinplot("Pclass", "Age", hue = "Survived", data = df_train, scale = 'count', split = True, ax = ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot("Sex", "Age", hue = "Survived", data = df_train, scale = 'count', split = True, ax = ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

왼쪽 그림은 Pclass 별로 Age의 distribution 이 어떻게 다른지, 거기에 생존여부에 따라 구분한 그래프입니다.  
오른쪽 그림도 마찬가지 Sex, 생존에 따른 distribution 이 어떻게 다른지 보여주는 그래프입니다.  
생존만 봤을 때, 모든 클래스에서 나이가 어릴 수록 생존을 많이 한것을 볼 수 있습니다.  
오른쪽 그림에서 보면, 명확히 여자가 생존을 많이 한것을 볼 수 있습니다.  
여성과 아이를 먼저 챙긴 것을 볼 수 있습니다.

## 2.6 Embarked
Embarked 는 탑승한 항구를 나타냅니다.  
위에서 해왔던 것과 비슷하게 탑승한 곳에 따르 생존률을 보겠습니다.

In [368]:
f, ax = plt.subplots(1,1,figsize=(7,7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = True).mean().sort_values(by='Survived', ascending = False).plot.bar(ax=ax)

보시다시피, 조금의 차이는 있지만 생존률은 좀 비슷한 거 같습니다. 그래도 C가 제일 높군요.  
모델에 얼마나 큰 영향을 미칠지는 모르겠지만, 그래도 사용하겠습니다.  
사실, 모델을 만들고 나면 우리가 사용한 feature 들이 얼마나 중요한 역할을 했는지 확인해볼 수 있습니다. 이는 추후에 모델을 만들고 난 다음에 살펴볼 것입니다.  
다른 feature 로 split 하여 한번 살펴보겠습니다.  

In [369]:
f, ax = plt.subplots(2,2,figsize = (20,15))
sns.countplot('Embarked', data = df_train, ax = ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')
sns.countplot('Embarked', hue = 'Sex', data = df_train, ax = ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked', hue = 'Survived', data = df_train, ax = ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked', hue = 'Pclass', data = df_train, ax = ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

- Figure(1) - 전체적으로 봤을 때, S 에서 가장 많은 사람이 탑승했습니다.  
- Figure(2) - C와 Q 는 남녀의 비율이 비슷하고, S는 남자가 더 많습니다.  
- Figure(3) - 생존확률이 S 경우 많이 낮은 걸 볼 수 있습니다. (이전 그래프에서 봤었습니다)  
- Figure(4) - Class 로 split 해서 보니, C가 생존확률이 높은건 클래스가 높은 사람이 많이 타서 그렇습니다. S는 3rd class 가 많아서 생존확률이 낮게 나옵니다.  

## 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)
SibSp와 Parch를 합하면 Family 가 될 것입니다. Family 로 합쳐서 분석해봅시다

In [370]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
# 자신을 포함해야하니 1을 더한다.
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

In [371]:
print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Minimum size of Family: ", df_train['FamilySize'].min())

FamilySize 와 생존의 관계를 한번 살펴봅시다

In [372]:
f, ax = plt.subplots(1,3,figsize=(40,10))
sns.countplot('FamilySize', data = df_train, ax = ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y = 1.02)
sns.countplot('FamilySize', hue = 'Survived', data = df_train, ax = ax[1])
ax[1].set_title('(2) Survived countplot dep ending on FamilySize', y=1.02)
df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index = True).mean().sort_values(by = 'Survived', ascending = False).plot.bar(ax = ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y = 1.02)

plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

- Figure (1) - 가족크기가 1-11까지 있음을 볼 수 있습니다. 대부분 1명이고 그 다음으로 2, 3, 4명입니다.  
- Figure (2), (3) - 가족 크기에 따른 생존비교입니다. 가족이 4명인 경우가 가장 생존확률이 높습니다. 가족수가 많아질수록, (5, 6, 7, 8, 11) 생존확률이 낮아지네요. 가족수가 너무 작아도(1), 너무 커도(5, 6, 8, 11) 생존 확률이 작네요. 3-4명 선에서 생존확률이 높은 걸 확인할 수 있습니다.

## 2.8 Fare
Fare 는 탑승요금이며, contious feature 입니다. 한번 histogram 을 그려보겠습니다.

In [373]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax = ax)
g = g.legend(loc = 'best')

그려진 그래프를 보면, distribution이 매우 비대칭이다.(high skewness)  
만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있다.  
몇개 없는 outlier 에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있다.  

outlier의 영향을 줄이기 위해 Fare 에 log 를 취하겠습니다.  
여기서 우리는 pandas 의 유용한 기능을 사용할 겁니다. dataFrame 의 특정 columns 에 공통된 작업(함수)를 적용하고 싶으면 아래의 map, 또는 apply 를 사용하면 매우 손쉽게 적용할 수 있습니다.  
우리가 지금 원하는 것은 Fare columns 의 데이터 모두를 log 값 취하는 것인데, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map 에 인수로 넣어주면, Fare columns 데이터에 그대로 적용이 됩니다. (매우 유용한 기능이니 꼭 숙지!)  

In [374]:
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()
# testset에 있는 nan value를 평균값으로 치환한다.

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [375]:
fig, ax = plt.subplots(1,1,figsize = (8,8))
g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax = ax )
g = g.legend(loc = 'best')

log 를 취하니, 이제 비대칭성이 많이 사라진 것을 볼 수 있습니다.  
우리는 이런 작업을 사용해 모델이 좀 더 좋은 성능을 내도록 할 수 있습니다.  
사실 방금한 것은 feature engineering 에 들어가는 부분인데, 여기서 작업했습니다.  
모델을 학습시키기 위해, 그리고 그 모델의 성능을 높이기 위해 feature 들에 여러 조작을 가하거나, 새로운 feature를 추가하는 것을 feature engineering 이라고 하는데, 우리는 이제 그것을 살펴볼 것입니다.

## 2.9 Cabin
이 feature 는 NaN 이 대략 80% 이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지는 않습니다.  
그러므로 우리가 세우려는 모델에 포함시키지 않도록 하겠습니다.

In [376]:
df_train.head()

## 2.10 Ticket
이 feature 는 NaN 은 없습니다.  
일단 string data 이므로 우리가 어떤 작업들을 해주어야 실제 모델에 사용할 수 있는데, 이를 위해선 사실 아이디어가 필요합니다.

In [377]:
df_train['Ticket'].value_counts()

# 3. Feature Engineering
- 본격적인 feature engineering 을 시작해보겠습니다.  
- 가장 먼저, dataset 에 존재하는 null data를 채우려고 합니다.  
- 아무 숫자로 채울 수는 없고, null data 를 포함하는 feature 의 statistics 를 참고하거나, 다른 아이디어를 짜내어 채울 수 있습니다.  
- null data 를 어떻게 채우느냐에 따라 모델의 성능이 좌지우지될 수 있기 때문에, 신경써줘야할 부분입니다.  
- Feature engineering 은 실제 모델의 학습에 쓰려고 하는 것이므로, train 뿐만아닐라 test 도 똑같이 적용해주어야 합니다. 잊지맙시다.  

## 3.1 Fill Null
### 1. Fill Null in Age Using Title
- Age 에는 null data가 177개나 있습니다. 이를 채울 수 있는 여러 아이디어가 있을 것인데, 여기서 우리는 title + statistics 를 사용해 보겠습니다.  
- 영어에서는 Miss, Mrr, Mrs 같은 title이 존재합니다. 각 탑승객의 이름에는 꼭 이런 title 이 들어가게 되는데 이를 사용해보겠습니다.  
- pandas series 에는 data 를 string 으로 바꿔주는 str method, 거기에 정규표현식을 적용하게 해주는 extract method가 있습니다. 이를 사용하여 title 을 쉽게 추출할 수 있습니다. title을 Initial column에 저장하겠습니다

In [378]:
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')   # Let's extract the Salutations
df_test['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.')     # Let's extract the Salutations

In [379]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')
# Checking the Initials with the Sex

위 테이블을 참고하여, 남자, 여자가 쓰는 initial을 구분해보겠습니다.  
replace 메소드를 사용하면, 특정 데이터 값을 원하는 값으로 치환해줍니다.

In [380]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'], 
                            ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)
df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'], 
                           ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [381]:
df_train.groupby('Initial').mean()

여성과 관계있는 Miss, Mr, Mrs가 생존률이 높은 것을 볼 수 있습니다.

In [382]:
df_train.groupby('Initial')['Survived'].mean().plot.bar()

이제 본격적으로 Null 을 채울 것입니다.  
null data 를 채우는 방법은 정말 많이 존재합니다.  
statistics 를 활용하는 방법도 있고, null data 가 없는 데이터를 기반으로 새로운 머신러닝 알고리즘을 만들어 예측해서 채워넣는 방식도 있습니다.  
여기서는 statistics 를 활용하는 방법을 사용할 것입니다.  

여기서 statistics 는 train data 의 것을 의미합니다.  
우리는 언제나 test 를 unseen 으로 둔 상태로 놔둬야 하며, train 에서 얻은 statistics 를 기반으로 test 의 null data 를 채워줘야 합니다.

In [383]:
df_train.groupby('Initial').mean()

- Age의 평균을 이용해 Null value 를 채우도록 하겠습니다.  
- pandas dataframe 을 다룰 때에는 boolean array를 이용해 indexing 하는 방법이 참으로 편리합니다.  
- 아래 코드 첫줄을 해석하자면, isnull() 이면서 Initial 이 Mr 인 조건을 만족하는 row(탑승객) 의 'Age' 의 값을 33으로 치환한다 입니다.  
- loc + boolean + column 을 사용해 값을 치환하는 방법은 자주 쓰이므로 꼭 익숙해집시다.  

In [384]:
df_train.loc[(df_train['Age'].isnull()) & (df_train['Initial']=='Mr')]

In [385]:
df_train.loc[(df_train['Age'].isnull()) & (df_train['Initial']=='Mr'), 'Age']

In [386]:
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Mr'),'Age']

In [387]:
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Mr'),'Age'] = 33
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Mrs'),'Age'] = 36
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Master'),'Age'] = 5
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Miss'),'Age'] = 22
df_train.loc[(df_train.Age.isnull()) & (df_train.Initial == 'Other'),'Age'] = 46

df_test.loc[(df_test.Age.isnull()) & (df_test.Initial == 'Mr'),'Age'] = 33
df_test.loc[(df_test.Age.isnull()) & (df_test.Initial == 'Mrs'),'Age'] = 36
df_test.loc[(df_test.Age.isnull()) & (df_test.Initial == 'Master'),'Age'] = 5
df_test.loc[(df_test.Age.isnull()) & (df_test.Initial == 'Miss'),'Age'] = 22
df_test.loc[(df_test.Age.isnull()) & (df_test.Initial == 'Other'),'Age'] = 46

In [388]:
df_train

### 2. Fill Null in Embarked

In [389]:
print('Embarked has ', sum(df_train['Embarked'].isnull()),'Null values')

- Embarked 는 Null value 가 2개이고, S 에서 가장 많은 탑승객이 있었으므로, 간단하게 Null 을 S로 채우겠습니다.
- dataframe 의 fillna method 를 이용하면 쉽게 채울 수 있습니다. 여기서 inplace=True 로 하면 df_train 에 fillna 를 실제로 적용하게 됩니다

In [390]:
df_train['Embarked'].fillna('S',inplace = True)

In [391]:
# 영상 추가 내용
df_train['Embarked'].isnull().sum()

## 3.2 Change Age (Continuous to Categorical)

Age 는 현재 continuous feature 입니다.  
이대로 써도 모델을 세울 수 있지만, Age 를 몇개의 group 으로 나누어 category 화 시켜줄 수 도 있습니다.  
continuous 를 categorical 로 바꾸면 자칫 information loss가 생길 수도 있습니다만, 본 튜토리얼에서는 다양한 방법을 소개하는 것이 목적이므로 진행하도록 하겠습니다.

방법은 여러가지가 있습니다. dataframe 의 indexing 방법인 loc 를 사용하여 직접해줄 수 있고, 아니면 apply 를 사용해 함수를 넣어줄 수 있습니다.  
첫번째로 loc 를 사용한 방법입니다. loc 는 자주쓰게 되므로 그 사용법을 숙지하시면 좋습니다.  
나이는 10살 간격으로 나누겠습니다.

In [392]:
# 영상 추가 내용
df_train['Age_cat'] = 0

In [393]:
df_train.head()

In [394]:
df_train['Age_cat'] = 0
df_train.loc[df_train['Age'] < 10, 'Age_cat'] = 0
df_train.loc[(10 <= df_train['Age']) & (df_train['Age'] < 20), 'Age_cat'] = 1
df_train.loc[(20 <= df_train['Age']) & (df_train['Age'] < 30), 'Age_cat'] = 2
df_train.loc[(30 <= df_train['Age']) & (df_train['Age'] < 40), 'Age_cat'] = 3
df_train.loc[(40 <= df_train['Age']) & (df_train['Age'] < 50), 'Age_cat'] = 4
df_train.loc[(50 <= df_train['Age']) & (df_train['Age'] < 60), 'Age_cat'] = 5
df_train.loc[(60 <= df_train['Age']) & (df_train['Age'] < 70), 'Age_cat'] = 6
df_train.loc[(70 <= df_train['Age']), 'Age_cat'] = 7

In [395]:
df_test['Age_cat'] = 0
df_test.loc[df_test['Age'] < 10, 'Age_cat'] = 0
df_test.loc[(10 <= df_test['Age']) & (df_test['Age'] < 20), 'Age_cat'] = 1
df_test.loc[(20 <= df_test['Age']) & (df_test['Age'] < 30), 'Age_cat'] = 2
df_test.loc[(30 <= df_test['Age']) & (df_test['Age'] < 40), 'Age_cat'] = 3
df_test.loc[(40 <= df_test['Age']) & (df_test['Age'] < 50), 'Age_cat'] = 4
df_test.loc[(50 <= df_test['Age']) & (df_test['Age'] < 60), 'Age_cat'] = 5
df_test.loc[(60 <= df_test['Age']) & (df_test['Age'] < 70), 'Age_cat'] = 6
df_test.loc[(70 <= df_test['Age']), 'Age_cat'] = 7

두번째로 간단한 함수를 만들어 apply 메소드에 넣어주는 방법입니다.  
훨씬 수월합니다.

In [396]:
def category_age(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    elif x < 30:
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 6
    else:
        return 7    
    
df_train['Age_cat_2'] = df_train['Age'].apply(category_age)

두가지 방법이 잘 적용됬다면, 둘다 같은 결과를 내야합니다.  
이를 확인하기 위해 Series 간 boolean 비교 후 all() 메소드를 사용합시다.  
all() 메소드는 모든 값이 True 면 True, 하나라도 False 가 있으면 False 를 줍니다.

In [397]:
df_train['Age_cat'] == df_train['Age_cat_2']

In [398]:
df_train['Age_cat'][888]

In [399]:
df_train['Age_cat_2'][888]

In [400]:
df_train

In [401]:
print('1번 방법, 2번 방법 둘 다 같은 결과를 내면 True 줘야함 -> ', (df_train['Age_cat'] == df_train['Age_cat_2']).all())

보시다시피 True 입니다. 둘 중 편한 걸 선택하시면 됩니다.  
이제 중복되는 Age_cat 컬럼과 원래 컬럼 Age 를 제거하겠습니다.

In [402]:
df_train.drop(['Age', 'Age_cat_2'], axis=1, inplace=True)
df_test.drop(['Age'], axis=1, inplace=True)

## 3.3 Change Initial, Embarked and Sex (String to Numerical)
현재 Initial 은 Mr, Mrs, Miss, Master, Other 총 5개로 이루어져 있습니다.  
이런 카테고리로 표현되어져 있는 데이터를 모델에 인풋으로 넣어줄 때 우리가 해야할 것은 먼저 컴퓨터가 인식할 수 있도록 수치화 시켜야 합니다.  
map method 를 가지고 간단히 할 수 있습니다.  
사전 순서대로 정리하여 mapping 하겠습니다.  

In [403]:
df_train['Initial'] = df_train['Initial'].map({'Master':0, 'Miss':1, 'Mr':2, 'Mrs':3, 'Other':4})
df_test['Initial'] = df_test['Initial'].map({'Master':0, 'Miss':1, 'Mr':2, 'Mrs':3, 'Other':4})

Embarked 도 C, Q, S로 이루어져 있습니다. map 을 이용해 바꿔봅시다.  
그러기 앞서서, 특정 column 에 어떤 값들이 있는 지 확인해보는 방법을 잠깐 살펴보겠습니다.   
간단히 unique() 메소드를 쓰거나, value_counts() 를 써서 count 까지 보는 방법이 있습니다.  

In [404]:
df_train

In [405]:
# 영상 추가 내용
f, ax = plt.subplots(1,1,figsize=(7,7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = True).mean().sort_values(by='Survived', ascending = False).plot.bar(ax=ax)

In [412]:
df_train['Embarked'].isnull().any()

In [413]:
df_train['Initial'].isnull().any()

In [415]:
df_train['Sex'].isnull().any()

In [414]:
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = True).mean().sort_values(by='Survived')

In [ ]:
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = True).mean().sort_index()

In [417]:
f, ax = plt.subplots(2,2,figsize = (20,15))
sns.countplot('Embarked', data=df_train, ax = ax[0,0])
ax[0,0].set_title('(1) No. OF Passengers Boarded')

sns.countplot('Embarked', hue = 'Sex', data=df_train, ax = ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')

sns.countplot('Embarked', hue = 'Survived', data=df_train, ax = ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')

sns.countplot('Embarked', hue = 'Pclass', data=df_train, ax = ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [419]:
df_train['Initial'].unique()

In [420]:
df_train['Embarked'].unique()

In [421]:
df_train['Embarked'].value_counts()

위 두 방법을 사용해 Embarked가 S, C, Q 세가지로 이루어진 것을 볼 수 있습니다. 이제 map을 사용해봅시다

In [422]:
df_train['Embarked'] = df_train['Embarked'].map({'C':0, 'Q':1, 'S':2})
df_test['Embarked'] = df_test['Embarked'].map({'C':0, 'Q':1, 'S':2})

한번 Null 이 사라졌는지 확인해봅시다.  
Embarked Column만 가져온 것은 하나의 pandas의 Series 객체므로, isnull() 메소드를 사용해 Series의 값들이 null 인지 아닌지에 대한 boolean 값을 얻을 수 있습니다. 그리고 이것에 any() 를 사용하여, True 가 단하나라도 있을 시(Null이 한개라도 있을 시) True 를 반환해주게 됩니다.  
우리는 Null 을 S로 다 바꿔주었으므로 False 를 얻게 됩니다

In [423]:
df_train.head()

In [424]:
df_train['Embarked'].isnull().any()

In [425]:
df_train['Initial'].isnull().any()

In [426]:
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1})
df_test['Sex'] = df_test['Sex'].map({'female':0, 'male':1})

In [427]:
df_train['Sex'].isnull().any()

이제 각 feature 간의 상관관계를 한번 보려고 합니다.   
두 변수간의 Pearson correlation 을 구하면 (-1, 1) 사이의 값을 얻을 수 있습니다.  
-1로 갈수록 음의 상관관계, 1로 갈수록 양의 상관관계를 의미하며, 0은 상관관계가 없다는 것을 의미합니다.  
구하는 수식은 아래와 같습니다.  
$$r_{xy} = \frac{Cov(x,y)}{S_{x}S_{y}} = \frac{\frac{1}{n-1}\sum_{i=1}^n(x_{i}-\bar x)(y_{i}-\bar y)}{S_{x}S_{y}}$$  
우리는 여러 feature 를 가지고 있으니 이를 하나의 maxtrix 형태로 보면 편할 텐데, 이를 heatmap plot 이라고 하며, dataframe 의 corr() 메소드와 seaborn 을 가지고 편하게 그릴 수 있습니다.

In [428]:
heatmap_data = df_train[['Survived', 'Pclass','Sex','Fare','Embarked','FamilySize','Initial','Age_cat']]

colormap = plt.cm.RdBu
plt.figure(figsize = (14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths = 0.1, vmax = 1.0, square = True, 
            cmap = colormap, linecolor = 'white', annot=True, annot_kws={"size": 16})

# del heatmap_data

우리가 EDA에서 살펴봤듯이, Sex 와 Pclass 가 Survived 에 상관관계가 어느 정도 있음을 볼 수 있습니다.  
생각보다 fare 와 Embarked 도 상관관계가 있음을 볼 수 있습니다.  
또한 우리가 여기서 얻을 수 있는 정보는 서로 강한 상관관계를 가지는 feature들이 없다는 것입니다.  
이것은 우리가 모델을 학습시킬 때, 불필요한(redundant, superfluous) feature 가 없다는 것을 의미합니다.  
1 또는 -1 의 상관관계를 가진 feature A, B 가 있다면, 우리가 얻을 수 있는 정보는 사실 하나일 거니까요.  
이제 실제로 모델을 학습시키기 앞서서 data preprocessing (전처리)을 진행해보겠습니다. 거의 다 와갑니다 힙냅시다!  

## 3.4 One-hot encoding on Initial and Embarked
수치화시킨 카테고리 데이터를 그대로 넣어도 되지만, 모델의 성능을 높이기 위해 one-hot encoding을 해줄 수 있습니다.  
수치화는 간단히 Master == 0, Miss == 1, Mr == 2, Mrs == 3, Other == 4 로 매핑해주는 것을 말합니다.  
One-hot encoding 은 위 카테고리를 아래와 같이 (0, 1) 로 이루어진 5차원의 벡터로 나타내는 것을 말합니다.  
이러한 엑셀같은 테이블 데이터 작업을 직접 코딩할 수도 있지만, pandas 의 get_dummies 를 사용하여 쉽게 해결 할 수 있습니다.  
총 5개의 카테고리니, one-hot encoding 을 하고 나면 새로운 5개의 column 이 생겨납니다.  
Initial 을 prefix로 두어서 구분이 쉽게 만들어 줍니다.  

In [429]:
df_train = pd.get_dummies(df_train, columns = ['Initial'], prefix = 'Initial')
df_test = pd.get_dummies(df_test, columns = ['Initial'], prefix = 'Initial')

In [430]:
df_train.head()

보시다시피 오른쪽에 우리가 만들려고 했던 one-hot encoded columns 가 생성된 것이 보입니다.  
Embarked 에도 적용하겠습니다. Initial 때와 마찬가지로 one-hot encoding 을 사용해 표현하겠습니다.

In [431]:
df_train = pd.get_dummies(df_train, columns = ['Embarked'], prefix = 'Embarked')
df_test = pd.get_dummies(df_test, columns = ['Embarked'], prefix = 'Embarked')

아주 쉽게 one-hot encoding 을 적용했습니다.  
sklearn 로 Labelencoder + OneHotencoder 이용해도 one-hot encoding 이 가능합니다.  
다른 튜토리얼에서 한번 써보겠습니다. 여기서는 get_dummies 로 충분히 가능하기 때문에 get_dummies 만으로 끝내겠습니다.  
가끔 category 가 100개가 넘어가는 경우가 있습니다. 이때 one-hot encoding을 사용하면 column이 100개가 생겨, 학습시 매우 버거울 경우가 있습니다.   
이런 경우는 다른 방법을 사용하기도 하는데, 이는 다음에 한번 다뤄보겠습니다.

## 3.5 Drop columns
이제 책상을 치울 때 입니다. 필요한 columns 만 남기고 다 지웁시다.

In [432]:
df_train.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin'], axis=1, inplace=True)

In [433]:
df_train.head()

In [434]:
df_test.head()

보시다시피, train 의 Survived feature(target class)를 빼면 train, test 둘다 같은 columns 를 가진 걸 확인할 수 있습니다.

# 4. Building machine learning model and prediction using the trained model
이제 준비가 다 되었으니 sklearn 을 사용해 본격적으로 머신러닝 모델을 만들어 봅시다.

In [435]:
# importing all the required ML packages
from sklearn.ensemble import RandomForestClassifier     # 유명한 randomforestclassifier입니다.
from sklearn import metrics                             # 모델의 평가를 위해 씁니다.
from sklearn.model_selection import train_test_split    # training set을 쉽게 나눠주는 함수입니다.

#### Sklearn
Sklearn 은 머신러닝의 처음부터 끝까지가 다 있습니다.   
feature engineering, preprocessing, 지도 학습 알고리즘, 비지도 학습 알고리즘, 모델 평가, 파이프라인 등 머신러닝에 관련된 모든 작업들이 손쉬운 인터페이스로 구현되어 있습니다. 데이터 분석 + 머신러닝을 하고싶다면, 이 라이브러리는 반드시 숙지해야합니다.  
파이썬 라이브러리를 활용한 머신러닝(Introduction to machine larning with Python)책을 사서 공부하시길 매우 추천해드립니다.

지금 타이타닉 문제는 target class(survived)가 있으며, target class 는 0, 1로 이루어져 있으므로(binary) binary classfication 문제입니다.  
우리가 지금 가지고 있는 train set 의 survived를 제외한 input 을 가지고 모델을 최적화시켜서 각 샘플(탑승객)의 생존유무를 판단하는 모델을 만들어 냅니다.  
그 후 모델이 학습하지 않았던 test set 을 input 으로 주어서 test set 의 각 샘플(탑승객)의 생존 유무를 예측해봅니다.  

## 4.1 Preparation - Split dataset into train, valid, test set
가장 먼저, 학습에 쓰일 데이터와, target label(Survived)를 분리합니다.  
drop 을 사용해 간단히 할 수 있습니다.  

In [436]:
X_train = df_train.drop('Survived', axis = 1). values
target_label = df_train['Survived'].values
X_test = df_test.values

보통 train, test 만 언급되지만, 실제 좋은 모델을 만들기 위해서 우리는 valid set을 따로 만들어 모델 평가를 해봅니다.  
마치 축구대표팀이 팀훈련(train)을 하고 바로 월드컵(test)로 나가는 것이 아니라,  
팀훈련(train)을 한 다음 평가전(valid)를 거쳐 팀의 훈련 정도(학습정도)를 확인하고 월드컵(test)에 나가는 것과 비슷합니다.  
train_test_split 을 사용하여 쉽게 train 셋을 분리할 수 있습니다.  

In [437]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size = 0.3, random_state = 2018)

sklearn 에서는 여러 머신러닝 알고리즘을 지원해줍니다. : http://scikit-learn.org/stable/supervised_learning.html#supervised-learning  

본 튜토리얼에서는 랜덤포레스트 모델을 사용하도록 하겠습니다.  
랜덤포레스트는 결정트리기반 모델이며, 여러 결정 트리들을 앙상블한 모델입니다.  

#### 파라미터 & 파라미터 튜닝
각 머신러닝 알고리즘에는 여러 파라미터들이 있습니다.  
랜덤포레스트분류기도 n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf 등 여러 파라미터들이 존재합니다.  
이것들이 어떻게 세팅되냐에 따라 같은 데이터셋이라 하더라도 모델의 성능이 달라집니다.  
파라미터 튜닝은 시간, 경험, 알고리즘에 대한 이해 등이 필요합니다.  
결국 많이 써봐야 모델도 잘 세울 수 있는 것이죠. 그래서 캐글을 추천합니다. 여러 데이터셋을 가지고 모델을 이리저리 써봐야 튜닝하는 감이 생길테니까요!  

일단 지금은 튜토리얼이니 파라미터 튜닝은 잠시 제쳐두기로 하고, 기본 default 세팅으로 진행하겠습니다.  
모델 객체를 만들고, fit 메소드로 학습시킵니다.  
그런 후 valid set input 을 넣어주어 예측값(X_vld sample(탑승객)의 생존여부)를 얻습니다.  

## 4.2 Model generation and prediction¶

In [438]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

단 세줄만으로 여러분은 모델을 세우고, 예측까지 해봤습니다.  
자, 이제 모델의 성능을 한번 살펴보겠습니다.

In [440]:
print('총 {}명 중 {:.2f}% 정확도로 생존을 맞춤'.format(y_vld.shape[0], 100 * metrics.accuracy_score(prediction, y_vld)))

## 4.3 Feature importance 
학습된 모델은 feature importance 를 가지게 되는데, 우리는 이것을 확인하여 지금 만든 모델이 어떤 feature 에 영향을 많이 받았는 지 확인할 수 있습니다.  
쉽게 말해, 10 = 4x1 + 2x2 + 1*x3 을 생각하면, 우리는 x1이 결과값(10)에 큰 영향을 준다고 생각 할 수 있습니다.  
feature importance 는 4, 2, 1 을 이야기하며, x1이 가장 큰 값(4)를 가지므로, 이 모델에 가장 큰 영향을 미친다고 말할 수 있습니다.  
학습된 모델은 기본적으로 featureimportances 를 가지고 있어서 쉽게 그 수치를 얻을 수 있습니다.  
pandas series 를 이용하면 쉽게 sorting 을 하여 그래프를 그릴 수 있습니다.  

In [442]:
from pandas import Series

feature_importance = model.feature_importances_
Series_feat_imp = Series(feature_importance, index=df_test.columns)

In [443]:
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

우리가 얻은 모델에서는 Fare 가 가장 큰 영향력을 가지며, 그 뒤로 Initial_2, Age_cat, Pclass가 차례로 중요도를 가집니다.  
사실 feature importance 는 지금 모델에서의 importance 를 나타냅니다.  
만약 다른 모델을 사용하게 된다면 feature importance 가 다르게 나올 수 있습니다.  
이 feature importance 를 보고 실제로 Fare 가 중요한 feature 일 수 있다고 판단을 내릴 수는 있지만, 이것은 결국 모델에 귀속되는 하나의 결론이므로 통계적으로 좀 더 살펴보긴 해야합니다.  
featuure importance 를 가지고 좀 더 정확도가 높은 모델을 얻기 위해 feature selection 을 할 수도 있고, 좀 더 빠른 모델을 위해 feature 제거를 할 수 있습니다.

## 4.4 Prediction on Test set 
이제 모델이 학습하지 않았던(보지 않았던) 테스트셋을 모델에 주어서, 생존여부를 예측해보겠습니다.  
이 결과는 실제로 submission(제출용) 이므로 결과는 leaderboard 에서 확인할 수 있습니다.  
캐글에서 준 파일, gender_submission.csv 파일을 읽어서 제출 준비를 하겠습니다.

In [446]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')

In [447]:
submission.head()

In [448]:
prediction = model.predict(X_test)
submission['Survived'] = prediction

In [449]:
submission.to_csv('./First_kaggle_practice.csv', index=False)